**Chapter 6: Developing Software with Generative AI**

In [5]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

from config import set_environment
set_environment()

## Writing code with LLMs

### StarCoder

### Small local model

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
checkpoint = "Salesforce/codegen-350M-mono"
model = AutoModelForCausalLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=500
)

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/797M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [2]:
text = """
def calculate_primes(n):
    \"\"\"Create a list of consecutive integers from 2 up to N.
    For example:
    >>> calculate_primes(20)
    Output: [2, 3, 5, 7, 11, 13, 17, 19]
\"\"\"
"""

In [3]:
completion = pipe(text)
print(completion[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



def calculate_primes(n):
    """Create a list of consecutive integers from 2 up to N.
    For example:
    >>> calculate_primes(20)
    Output: [2, 3, 5, 7, 11, 13, 17, 19]
"""
    p = 3
    k = 1
    p_k = []
    while k < n:
        if is_prime(p):
            p_k.append(p)
            k += 1
        p += 1
    return p_k


def is_prime(n):
    """If n is a prime number, then return true. Otherwise return false.
    >>> is_prime(34)
    True
    >>> is_prime(37)
    False
    """
    if n <= 1:
        raise ValueError("n must be greater than 1")
    for i in range(2, round(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True


def count_conv(n):
    """Count all divisors of prime numbers up to number N.
    >>> count_conv(20)
    Output: 9
    >>> count_conv(24)
    Output: 11
    """
    if is_prime(n):
        return 1
    k = 2
    primes = []
    for n in range(2, n + 1):
        if is_prime(n):
            primes.append(n)
            k += 1
    retu

## Automating software development

In [7]:
from langchain.llms.openai import OpenAI
from langchain.agents import load_tools, initialize_agent, AgentType
llm = OpenAI(model="gpt-3.5-turbo-instruct")
tools = load_tools(["python_repl"])
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
result = agent("What are the prime numbers until 20?")
print(result)



> Entering new AgentExecutor chain...
 I need to find out what a prime number is
Action: print(...)
Action Input: 20
Observation: print(...) is not a valid tool, try one of [Python_REPL].
Thought:

Python REPL can execute arbitrary code. Use with caution.


 A prime number is a number that is only divisible by 1 and itself
Action: Python_REPL
Action Input: 2
Observation: 
Thought: 2 is a prime number
Action: Python_REPL
Action Input: 3
Observation: 
Thought: 3 is a prime number
Action: Python_REPL
Action Input: 4
Observation: 
Thought: 4 is not a prime number
Action: Python_REPL
Action Input: 5
Observation: 
Thought: 5 is a prime number
Action: Python_REPL
Action Input: 6
Observation: 
Thought: 6 is not a prime number
Action: Python_REPL
Action Input: 7
Observation: 
Thought: 7 is a prime number
Action: Python_REPL
Action Input: 8
Observation: 
Thought: 8 is not a prime number
Action: Python_REPL
Action Input: 9
Observation: 
Thought: 9 is not a prime number
Action: Python_REPL
Action Input: 10
Observation: 
Thought: 10 is not a prime number
Action: Python_REPL
Action Input: 11
Observation: 
Thought: 11 is a prime number
Action: Python_REPL
Action Input: 12
Observation: 
Thought: 12 is not a prime number
Action: Python_REPL
Action Input: 

In [ ]:
from langchain import OpenAI
from langchain_experimental.plan_and_execute import load_chat_planner, load_agent_executor, PlanAndExecute

llm = OpenAI()
planner = load_chat_planner(llm)
executor = load_agent_executor(
    llm,
    tools,
    verbose=True,
)
agent_executor = PlanAndExecute(
    planner=planner,
    executor=executor,
    verbose=True,
    handle_parsing_errors="Check your output and make sure it conforms!",
    return_intermediate_steps=True
)
agent_executor.run("Write a tetris game in python!")

In [8]:
from langchain import PromptTemplate, LLMChain, OpenAI

DEV_PROMPT = (
    "You are a software engineer who writes Python code given tasks or objectives. "
    "Come up with a python code for this task: {task}"
    "Please use PEP8 syntax and comments!"
)
software_prompt = PromptTemplate.from_template(DEV_PROMPT)
software_llm = LLMChain(
    llm=OpenAI(
        temperature=0,
        max_tokens=1000
    ),
    prompt=software_prompt
)

In [ ]:
class PythonDeveloper():
    """Execution environment for Python code."""
    
    def __init__(
        self,
        llm_chain: Chain
    ):
        self.llm_chain = llm_chain
        
    def write_code(self, task: str) -> str:
        return self.llm_chain.run(task)
        
    def run(
        self,
        task: str,
    ) -> str:
        """Generate and Execute Python code."""
        code = self.write_code(task)
        try:
            return self.execute_code(code, "main.py")
        except Exception as ex:
            return str(ex)
            
    def execute_code(self, code: str, filename: str) -> str:
        """Execute a python code."""
        try:
            with set_directory(Path(self.path)):
                ns = dict(__file__=filename, __name__="__main__")
                function = compile(code, "<>", "exec")
                with redirect_stdout(io.StringIO()) as f:
                    exec(function, ns)
                    return f.getvalue()

In [ ]:
from langchain.tools import Tool
from software_development.python_developer import PythonDeveloper, PythonExecutorInput

software_dev = PythonDeveloper(llm_chain=software_llm)
code_tool = Tool.from_function(
    func=software_dev.run,
    name="PythonREPL",
    description=(
        "You are a software engineer who writes Python code given a function description or task."
    ),
    args_schema=PythonExecutorInput
)